In [1]:
import pandas as pd
import shutil
import numpy as np
import os
import tensorflow as tf

def move(src, dest):
    if os.path.isfile(src):
        shutil.move(src, dest)

def append_extension(fn):
    return (fn + ".jpg").zfill(7)
    

DATADIR = r"./adult - Copy"

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(
            len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs",
        )
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


# CSV_PATH = r"./adult/CastControls_ALP.xlsx"
# response = pd.read_excel(CSV_PATH, sheet_name=0,)[["GreenID", "Grade"]].dropna(
#     axis=0, subset=["Grade"]
# )
# response.Grade = response.Grade.astype("int")
# response.GreenID = response.GreenID.astype("str").apply(append_extension)
# response = response[response.Grade != 99]
# response = pd.concat(
#     [response, pd.DataFrame.from_dict(dict(response.Grade.apply(ordered_logit))).T,],
#     axis=1,
# )
# response.apply(lambda row: move(DATADIR + '/' + str(row['GreenID']), DATADIR + '/' + str(row['Grade']) + '/'+ str(row['GreenID'])), axis=1)

1 Physical GPUs, 1 Logical GPUs


In [2]:
list_ds = tf.data.Dataset.list_files(str(DATADIR+'*/*'))
for f in list_ds.take(5):
    print(f.numpy())

b'.\\adult - Copy\\5\\106.jpg'
b'.\\adult - Copy\\3\\302.jpg'
b'.\\adult - Copy\\3\\153.jpg'
b'.\\adult - Copy\\6\\892.jpg'
b'.\\adult - Copy\\3\\446.jpg'


In [3]:
@tf.function
def ordered_logit(class_number):
    # zero portability
    target = np.zeros(4, dtype=int)
    target[: class_number - 2] = 1
    return target


def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    return [int(parts[-2])]

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [380, 380])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)